In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

##ELT

In [ ]:
conn = sqlite3.connect("olist_warehouse.db")

In [ ]:
items = pd.read_csv('https://drive.google.com/uc?id=1YAIZ1v2GZPE5epki5dsCdVp5zQLSZaEl')
payments = pd.read_csv('https://drive.google.com/uc?id=1OhNps6JLk5ttkVeNN5PvApDF4_m7l_JN')
reviews = pd.read_csv('https://drive.google.com/uc?id=1Y72vau9ATI62bI9V9MiSVF9C-nV49aVE')
customers = pd.read_csv('https://drive.google.com/uc?id=1eKoY4f2iRseaouKZgAl3JEdVZslZKrZ5')
orders = pd.read_csv('https://drive.google.com/uc?id=1QArWoteWXvlftx5Ra4ZXyg8YM18WDBTs')
products = pd.read_csv('https://drive.google.com/uc?id=1UcWev0Fb3CxGdGmeLxAJ2TLiL5JsFuym')
category = pd.read_csv('https://drive.google.com/uc?id=1jTV_ASCVRQXN2jFLomXp36toKW3cORgN')

In [ ]:
items.to_sql("raw_order_items", conn, if_exists="replace", index=False)
payments.to_sql("raw_order_payments", conn, if_exists="replace", index=False)
reviews.to_sql("raw_order_reviews", conn, if_exists="replace", index=False)
customers.to_sql("raw_customers", conn, if_exists="replace", index=False)
orders.to_sql("raw_orders", conn, if_exists="replace", index=False)
products.to_sql("raw_products", conn, if_exists="replace", index=False)
category.to_sql("raw_product_category_translation", conn, if_exists="replace", index=False)

71

In [ ]:
conn.execute("ALTER TABLE raw_order_items ADD COLUMN load_timestamp DATETIME;")
conn.execute("ALTER TABLE raw_order_payments ADD COLUMN load_timestamp DATETIME;")
conn.execute("ALTER TABLE raw_order_reviews ADD COLUMN load_timestamp DATETIME;")
conn.execute("ALTER TABLE raw_customers ADD COLUMN load_timestamp DATETIME;")
conn.execute("ALTER TABLE raw_orders ADD COLUMN load_timestamp DATETIME;")
conn.execute("ALTER TABLE raw_products ADD COLUMN load_timestamp DATETIME;")
conn.execute("ALTER TABLE raw_product_category_translation ADD COLUMN load_timestamp DATETIME;")
conn.commit()

In [ ]:
conn.execute("UPDATE raw_order_items SET load_timestamp = CURRENT_TIMESTAMP;")
conn.execute("UPDATE raw_order_payments SET load_timestamp = CURRENT_TIMESTAMP;")
conn.execute("UPDATE raw_order_reviews SET load_timestamp = CURRENT_TIMESTAMP;")
conn.execute("UPDATE raw_customers SET load_timestamp = CURRENT_TIMESTAMP;")
conn.execute("UPDATE raw_orders SET load_timestamp = CURRENT_TIMESTAMP;")
conn.execute("UPDATE raw_products SET load_timestamp = CURRENT_TIMESTAMP;")
conn.execute("UPDATE raw_product_category_translation SET load_timestamp = CURRENT_TIMESTAMP;")
conn.commit()

In [ ]:
# conn.execute("DROP TABLE IF EXISTS raw_order_reviews;")
# conn.commit()

In [ ]:
conn.execute("""
CREATE TABLE clean_payments AS
SELECT
    order_id,
    SUM(payment_value) AS total_payment_value
FROM raw_order_payments
WHERE payment_value IS NOT NULL
GROUP BY order_id;
""")

In [ ]:
# conn.execute("DROP TABLE IF EXISTS fact_orders;")
# conn.commit()

In [ ]:
pd.read_sql_query("PRAGMA table_info(order_month);", conn)

,cid,name,type,notnull,dflt_value,pk


In [ ]:
conn.execute("""
CREATE TABLE clean_reviews AS
SELECT
    order_id,
    review_score,
    CASE
        WHEN review_score >= 4 THEN 'good'
        ELSE 'bad'
    END AS review_category,
    CAST(strftime('%m', review_creation_date) AS INTEGER) AS order_month
FROM raw_order_reviews
WHERE review_score IS NOT NULL;
""")


In [ ]:
conn.execute("""
CREATE TABLE clean_customers AS
SELECT
    customer_id,
    customer_unique_id,
    customer_city,
    customer_state
FROM raw_customers;
""")

In [ ]:
conn.execute("""
CREATE TABLE order_value AS
SELECT
    order_id,
    SUM(price) AS total_order_value
FROM raw_order_items
GROUP BY order_id;
""")

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS clean_orders;
""")

conn.execute("""
CREATE TABLE clean_orders AS
SELECT
    order_id,
    customer_id,
    order_purchase_timestamp
FROM raw_orders;
""")

conn.commit()

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS feat_order_value;
""")

conn.execute("""
CREATE TABLE feat_order_value AS
SELECT
    order_id,
    SUM(price) AS total_order_value
FROM raw_order_items
GROUP BY order_id;
""")

conn.commit()

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS feat_item_features;
""")

conn.execute("""
CREATE TABLE feat_item_features AS
SELECT
    order_id,
    product_id,
    price,
    freight_value,

    -- Feature 1
    CASE
        WHEN price > 0 THEN freight_value / price
        ELSE 0
    END AS shipping_ratio,

    -- Feature 2
    price / (SELECT MAX(price) FROM raw_order_items) AS price_norm,

    -- Feature 3
    freight_value / (SELECT MAX(freight_value) FROM raw_order_items) AS freight_norm

FROM raw_order_items;
""")

conn.commit()

In [ ]:
conn.execute("""
CREATE TABLE dim_product AS
SELECT
    p.product_id,
    COALESCE(t.product_category_name_english, p.product_category_name)
        AS product_category_name
FROM raw_products p
LEFT JOIN raw_product_category_translation t
    ON p.product_category_name = t.product_category_name;
""")

conn.commit()

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS fact_orders;""")
conn.execute("""
CREATE TABLE fact_orders AS
SELECT
    f.order_id,
    f.product_id,
    ord.customer_id,
    ord.order_purchase_timestamp,
    f.price,
    f.freight_value,
    ov.total_order_value,
    f.shipping_ratio,
    f.price_norm,
    f.freight_norm,
    p.total_payment_value,
    r.review_score,
    r.review_category,
    r.order_month,
    c.customer_city,
    c.customer_state,
    dp.product_category_name

FROM feat_item_features f
LEFT JOIN feat_order_value ov
    ON f.order_id = ov.order_id
LEFT JOIN clean_orders ord
    ON f.order_id = ord.order_id
LEFT JOIN clean_payments p
    ON f.order_id = p.order_id
LEFT JOIN clean_reviews r
    ON f.order_id = r.order_id
LEFT JOIN clean_customers c
    ON ord.customer_id = c.customer_id
LEFT JOIN dim_product dp
    ON f.product_id = dp.product_id;

""")

conn.commit()


In [ ]:
pd.read_sql_query("SELECT * FROM fact_orders LIMIT 10;", conn)

,order_id,product_id,customer_id,order_purchase_timestamp,price,freight_value,total_order_value,shipping_ratio,price_norm,freight_norm,total_payment_value,review_score,review_category,order_month,customer_city,customer_state,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,58.90,13.29,58.90,0.225637,0.008745,0.032440,72.19,5,good,9,campos dos goytacazes,RJ,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,239.90,19.93,239.90,0.083076,0.035620,0.048648,259.83,4,good,5,santa fe do sul,SP,pet_shop
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,199.00,17.87,199.00,0.089799,0.029547,0.043619,216.87,5,good,1,para de minas,MG,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,12.99,12.79,12.99,0.984604,0.001929,0.031219,25.78,4,good,8,atibaia,SP,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,199.90,18.14,199.90,0.090745,0.029681,0.044278,218.04,5,good,3,varzea paulista,SP,garden_tools
5,00048cc3ae777c65dbb7d2a0634bc1ea,ef92defde845ab8450f9d70c526ef70f,816cbea969fe5b689b39cfc97a506742,2017-05-15 21:42:34,21.90,12.69,21.90,0.579452,0.003252,0.030975,34.59,4,good,5,uberaba,MG,housewares
6,00054e8431b9d7675808bcb819fb4a32,8d4f2bb7e93e6710a28f34fa83ee7d28,32e2e6ab09e778d99bf2e0ecd4898718,2017-12-10 11:53:48,19.90,11.85,19.90,0.595477,0.002955,0.028925,31.75,4,good,12,guararapes,SP,telephony
7,000576fe39319847cbb9d288c5617fa6,557d850972a7d6f792fd18ae1400d9b6,9ed5e522dd9dd85b4af4a077526d8117,2018-07-04 12:08:27,810.00,70.75,810.00,0.087346,0.120267,0.172696,880.75,5,good,7,praia grande,SP,garden_tools
8,0005a1a1728c9d785b8e2b08b904576c,310ae3c140ff94b03219ad0adc3c778f,16150771dfd4776261284213b89c304e,2018-03-19 18:40:33,145.95,11.65,145.95,0.079822,0.021670,0.028437,157.60,1,bad,3,santos,SP,health_beauty
9,0005f50442cb953dcd1d21e1fb923495,4535b0e1091c278dfd193e5a1d63b39f,351d3cb2cee3c7fd0af6616c82df21d3,2018-07-02 13:59:39,53.99,11.40,53.99,0.211150,0.008016,0.027827,65.39,4,good,7,jandira,SP,books_technical


In [ ]:
pd.read_sql_query("""SELECT
    strftime('%Y-%m', ord.order_purchase_timestamp) AS sales_month,
    SUM(i.price) AS total_revenue,
    COUNT(DISTINCT i.order_id) AS total_orders
FROM raw_order_items i
JOIN raw_orders ord ON i.order_id = ord.order_id
GROUP BY sales_month
ORDER BY sales_month;""", conn)

,sales_month,total_revenue,total_orders
0,2016-09,267.36,3
1,2016-10,49507.66,308
2,2016-12,10.90,1
3,2017-01,120312.87,789
4,2017-02,247303.02,1733
5,2017-03,374344.30,2641
6,2017-04,359927.23,2391
7,2017-05,506071.14,3660
8,2017-06,433038.60,3217
9,2017-07,498031.48,3969


In [ ]:
pd.read_sql_query("""SELECT
    p.product_category_name,
    SUM(f.price) as total_revenue,
    COUNT(f.product_id) as quantity_sold
FROM fact_orders f
JOIN dim_product p ON f.product_id = p.product_id
GROUP BY p.product_category_name
ORDER BY total_revenue DESC
LIMIT 10;""", conn)

,product_category_name,total_revenue,quantity_sold
0,health_beauty,1263138.54,9727
1,watches_gifts,1206075.33,6001
2,bed_bath_table,1050936.61,11270
3,sports_leisure,993656.51,8700
4,computers_accessories,919640.54,7894
5,furniture_decor,736282.47,8415
6,cool_stuff,637258.51,3806
7,housewares,634542.60,6989
8,auto,594363.10,4256
9,garden_tools,486432.45,4361


In [ ]:
pd.read_sql_query("""
SELECT
    CAST(strftime('%H', order_purchase_timestamp) AS INTEGER) AS hour_of_day,
    COUNT(DISTINCT order_id) AS total_transactions
FROM fact_orders
GROUP BY hour_of_day
ORDER BY total_transactions DESC;
""", conn)

,hour_of_day,total_transactions
0,16,6628
1,11,6528
2,14,6521
3,13,6461
4,15,6397
5,21,6177
6,20,6144
7,10,6112
8,17,6105
9,12,5947


In [ ]:
pd.read_sql_query("""
SELECT
    customer_state,
    AVG(freight_value) AS avg_shipping_cost,
    AVG(shipping_ratio) AS avg_shipping_ratio
FROM fact_orders
GROUP BY customer_state
ORDER BY avg_shipping_cost DESC;
""", conn)

,customer_state,avg_shipping_cost,avg_shipping_ratio
0,RR,42.984423,0.589050
1,PB,42.694129,0.511206
2,RO,41.069712,0.594188
3,AC,40.073370,0.511651
4,PI,39.114954,0.499573
5,MA,38.197865,0.550576
6,TO,37.246603,0.499459
7,SE,36.653169,0.464946
8,AL,35.744018,0.475496
9,PA,35.742870,0.474183


In [ ]:
pd.read_sql_query("""
SELECT *
FROM fact_orders
WHERE price > (
    SELECT AVG(price) * 3
    FROM fact_orders
)
ORDER BY price DESC;
""", conn)

,order_id,product_id,customer_id,order_purchase_timestamp,price,freight_value,total_order_value,shipping_ratio,price_norm,freight_norm,total_payment_value,review_score,review_category,order_month,customer_city,customer_state,product_category_name
0,0812eb902a67711a1cb742b3cdaa65ae,489ae2aa008f021502940f251d4cce7f,c6e2731c5b391845f6800c97401a43a9,2017-02-12 20:37:36,6735.00,194.31,6735.00,0.028851,1.000000,0.474297,6929.31,5.0,good,3.0,campo grande,MS,housewares
1,fefacc66af859508bf1a7934eab1e97f,69c590f7ffc7bf8db97190b6cb6ed62e,f48d464a0baaea338cb25f816991ab1f,2018-07-25 18:10:17,6729.00,193.21,6729.00,0.028713,0.999109,0.471612,6922.21,NaN,None,NaN,vitoria,ES,computers
2,f5136e38d1a14a4dbd87dff67da82701,1bdf5e6731585cf01aa8169c7028d6ad,3fd6777bbce08a352fddd04e4a7cc8f6,2017-05-24 18:14:34,6499.00,227.66,6499.00,0.035030,0.964959,0.555702,6726.66,5.0,good,6.0,marilia,SP,art
3,a96610ab360d42a2e5335a3998b4718a,a6492cc69376c469ab6f61d8f44de961,df55c14d1476a9a3467f131269c2477f,2017-04-01 15:58:40,4799.00,151.34,4799.00,0.031536,0.712546,0.369410,4950.34,5.0,good,4.0,araruama,RJ,small_appliances
4,199af31afc78c699f0dbf71fb178d4d4,c3ed642d592594bb648ff4a04cee2747,24bbf5fd2f2e1b359ee7de94defc4a15,2017-04-18 18:50:13,4690.00,74.34,4690.00,0.015851,0.696362,0.181459,4764.34,4.0,good,5.0,maua,SP,small_appliances
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5276,a29fb245759386c446688eb1d7e07120,958cd4fc82ce0223c757f47bc9d67e23,bd6415726ba4ca2aeee0ab3ae08bd1cc,2018-04-11 10:47:51,362.45,17.55,362.45,0.048420,0.053816,0.042838,380.00,4.0,good,4.0,cacador,SC,auto
5277,a9196394f683e767c4c6f64dd7aeed82,958cd4fc82ce0223c757f47bc9d67e23,906240da12be29400c2914d820793e4f,2018-05-02 22:13:15,362.45,23.41,362.45,0.064588,0.053816,0.057142,385.86,5.0,good,5.0,sete lagoas,MG,auto
5278,5fa537c43d686925db746d574cbc2e0e,731509faf04a1b5119c24cc34f0d790e,4377320bd47b03d9a6fdc797da4a6cae,2017-03-14 15:19:58,361.99,13.14,361.99,0.036299,0.053748,0.032074,375.13,4.0,good,3.0,mogi das cruzes,SP,perfumery
5279,d0e379ef6e0ea864c470893dbd5da4dd,0573b564a90d2d9f9c460eb963da8c33,39d7c7a5e9c2172c16234017faf854ba,2018-03-11 19:07:43,361.98,47.81,672.22,0.132079,0.053746,0.116701,767.84,1.0,bad,3.0,campinas,SP,housewares


In [ ]:
# pd.read_sql_query("""
# SELECT order_id, price, customer_state
# FROM vw_high_value_orders
# ORDER BY price DESC
# LIMIT 10;
# """, conn)

,order_id,price,customer_state
0,0812eb902a67711a1cb742b3cdaa65ae,6735.00,MS
1,fefacc66af859508bf1a7934eab1e97f,6729.00,ES
2,f5136e38d1a14a4dbd87dff67da82701,6499.00,SP
3,a96610ab360d42a2e5335a3998b4718a,4799.00,RJ
4,199af31afc78c699f0dbf71fb178d4d4,4690.00,SP
5,8dbc85d1447242f3b127dda390d56e19,4590.00,PB
6,426a9742b533fc6fed17d1fd6d143d7e,4399.87,SP
7,68101694e5c5dc7330c91e1bbc36214f,4099.99,MG
8,b239ca7cd485940b31882363b52e6674,4059.00,MG
9,86c4eab1571921a6a6e248ed312f5a5a,3999.90,SP


In [ ]:
pd.read_sql_query("""
SELECT
    CASE
        WHEN CAST(strftime('%w', order_purchase_timestamp) AS INTEGER) IN (0, 6)
        THEN 'Weekend'
        ELSE 'Weekday'
    END AS day_type,
    SUM(price) AS revenue
FROM fact_orders
GROUP BY day_type;
""", conn)

,day_type,revenue
0,Weekday,1.054335e+07
1,Weekend,3.108569e+06


In [ ]:
pd.read_sql_query("""
SELECT
    customer_id,
    SUM(price) AS total_spent,
    COUNT(DISTINCT order_id) AS frequency
FROM fact_orders
GROUP BY customer_id
HAVING total_spent > 500
ORDER BY total_spent DESC;
""", conn)

,customer_id,total_spent,frequency
0,1617b1357756262bfa56ab541c47bc16,13440.00,1
1,ec5b2ba62e574342386871631fafd3fc,7160.00,1
2,c6e2731c5b391845f6800c97401a43a9,6735.00,1
3,f48d464a0baaea338cb25f816991ab1f,6729.00,1
4,3fd6777bbce08a352fddd04e4a7cc8f6,6499.00,1
...,...,...,...
3646,189c79f9536b271219661d456ac92076,503.34,1
3647,11ad10d9633e704794cda59de17627f1,503.34,1
3648,2f3dc6fda9c82bef02f23f9387ef137e,502.00,1
3649,a72282d3fc20f2e29b2659372f3aa616,501.90,1


In [ ]:
pd.read_sql_query("""
SELECT
    p.product_category_name,
    AVG(f.price) AS avg_item_price,
    MIN(f.price) AS min_price,
    MAX(f.price) AS max_price
FROM fact_orders f
JOIN dim_product p ON f.product_id = p.product_id
GROUP BY p.product_category_name;
""", conn)

,product_category_name,avg_item_price,min_price,max_price
0,None,111.834516,3.00,3980.00
1,agro_industry_and_commerce,342.124858,12.99,2990.00
2,air_conditioning,185.269226,10.90,1599.00
3,art,115.802105,3.50,6499.00
4,arts_and_craftmanship,75.583750,9.80,289.49
...,...,...,...,...
69,stationery,91.604044,2.29,1693.00
70,tablets_printing_image,90.703735,14.90,889.99
71,telephony,71.173495,5.00,2428.00
72,toys,117.519976,4.90,1699.99


In [ ]:
df_fact = pd.read_sql_query("SELECT * FROM fact_orders;", conn)
df_fact.to_csv("fact_orders.csv", index=False)

In [ ]:
from google.colab import files
files.download("fact_orders.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import files
# files.download("dashboard_all_metrics.csv")